In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls

MovieAnalysis.ipynb


In [3]:
#df_bom = pd.read_csv('bom.movie_gross.csv.gz')
df_imdb_name = pd.read_csv('../zippedData/imdb.name.basics.csv.gz')
imdb_title_akas = pd.read_csv('../zippedData/imdb.title.akas.csv.gz')
imdb_title_basics = pd.read_csv('../zippedData/imdb.title.basics.csv.gz')
df_imdb_title_crew = pd.read_csv('../zippedData/imdb.title.crew.csv.gz')
df_imdb_title_principals = pd.read_csv('../zippedData/imdb.title.principals.csv.gz')
df_imdb_title_ratings = pd.read_csv('../zippedData/imdb.title.ratings.csv.gz')
#do data cleaning on df_rt_movie_info, df_rt_reviews, tmdb
#df_rt_movie_info = pd.read_csv('rt.movie_info.tsv.gz', delimiter="\t", encoding='latin-1')
#df_rt_reviews = pd.read_csv('rt.reviews.tsv.gz', delimiter="\t", encoding='latin-1')
tmdb = pd.read_csv('../zippedData/tmdb.movies.csv.gz', index_col = 0)
#tn = pd.read_csv('tn.movie_budgets.csv.gz')

In [4]:
#df_bom.head()

In [5]:
#df_imdb_name.head()

In [6]:
#imdb_title_akas.head()

In [23]:
imdb_title_basics

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,NaN


In [8]:
#df_imdb_title_crew.head()

In [9]:
#df_imdb_title_principals.head()

In [10]:
#df_imdb_title_ratings.head()

In [11]:
#df_rt_movie_info.head()

In [12]:
#df_rt_reviews.tail()

In [13]:
tmdb.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [14]:
imdb_tmdb_merge = pd.merge(imdb_title_basics, tmdb, how='inner', on = 'original_title')
imdb_tmdb_merge

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,"[35, 18]",299782,en,9.800,2018-11-02,The Other Side of the Wind,7.0,64
1,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller","[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26
2,tt4503112,Bigfoot,Bigfoot,2018,NaN,"Action,Horror","[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26
3,tt9181914,Bigfoot,Bigfoot,2018,86.0,"Animation,Family","[27, 878, 12]",117856,en,2.813,2012-06-30,Bigfoot,2.9,26
4,tt0249516,Foodfight!,Foodfight!,2012,91.0,"Action,Animation,Comedy","[16, 28, 35, 10751]",116977,en,4.705,2013-05-07,Foodfight!,2.1,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21073,tt9678886,Vacancy,Vacancy,2018,81.0,Documentary,"[35, 27]",475066,en,0.649,2017-10-27,Vacancy,9.0,1
21074,tt9777830,John Leguizamo's Latin History for Morons,John Leguizamo's Latin History for Morons,2018,NaN,Comedy,[35],557606,en,4.304,2018-11-05,John Leguizamo's Latin History for Morons,7.3,13
21075,tt9814730,The Flare,The Flare,2017,63.0,Sport,[],469698,en,0.600,2017-07-11,The Flare,9.0,1
21076,tt9862978,Terra,Terra,2018,60.0,Documentary,[27],536235,en,1.767,2018-04-24,Terra,5.0,1


In [15]:
imdb_tmdb_merge.drop(columns = ['tconst', 'genre_ids', 'id', 'original_title', 'title', 'original_language'], axis=1, inplace = True)
imdb_tmdb_merge

,primary_title,start_year,runtime_minutes,genres,popularity,release_date,vote_average,vote_count
0,The Other Side of the Wind,2018,122.0,Drama,9.800,2018-11-02,7.0,64
1,Bigfoot,2017,NaN,"Horror,Thriller",2.813,2012-06-30,2.9,26
2,Bigfoot,2018,NaN,"Action,Horror",2.813,2012-06-30,2.9,26
3,Bigfoot,2018,86.0,"Animation,Family",2.813,2012-06-30,2.9,26
4,Foodfight!,2012,91.0,"Action,Animation,Comedy",4.705,2013-05-07,2.1,46
...,...,...,...,...,...,...,...,...
21073,Vacancy,2018,81.0,Documentary,0.649,2017-10-27,9.0,1
21074,John Leguizamo's Latin History for Morons,2018,NaN,Comedy,4.304,2018-11-05,7.3,13
21075,The Flare,2017,63.0,Sport,0.600,2017-07-11,9.0,1
21076,Terra,2018,60.0,Documentary,1.767,2018-04-24,5.0,1


In [19]:
imdb_tmdb_merge['start_year'] = imdb_tmdb_merge['start_year'].astype(int)
#imdb_tmdb_merge['start_year'] = pd.to_numeric(imdb_tmdb_merge['start_year'])
#imdb_tmdb_merge['start_year'] = int(imdb_tmdb_merge['start_year'])


type(imdb_tmdb_merge['start_year'][0])

numpy.int64

In [22]:
imdb_tmdb_merge = imdb_tmdb_merge[imdb_tmdb_merge['start_year'] >= 2000]
imdb_tmdb_merge

,primary_title,start_year,runtime_minutes,genres,popularity,release_date,vote_average,vote_count
0,The Other Side of the Wind,2018,122.0,Drama,9.800,2018-11-02,7.0,64
1,Bigfoot,2017,NaN,"Horror,Thriller",2.813,2012-06-30,2.9,26
2,Bigfoot,2018,NaN,"Action,Horror",2.813,2012-06-30,2.9,26
3,Bigfoot,2018,86.0,"Animation,Family",2.813,2012-06-30,2.9,26
4,Foodfight!,2012,91.0,"Action,Animation,Comedy",4.705,2013-05-07,2.1,46
...,...,...,...,...,...,...,...,...
21073,Vacancy,2018,81.0,Documentary,0.649,2017-10-27,9.0,1
21074,John Leguizamo's Latin History for Morons,2018,NaN,Comedy,4.304,2018-11-05,7.3,13
21075,The Flare,2017,63.0,Sport,0.600,2017-07-11,9.0,1
21076,Terra,2018,60.0,Documentary,1.767,2018-04-24,5.0,1


In [ ]:
type(tmdb['genre_ids'][0])

In [ ]:
#tmdb['vote_count'].value_counts()

In [ ]:
print(tmdb.shape)
print(tmdb.info())
print(tmdb.describe())

In [ ]:
genre_id_dict = {28: "Action", 12: "Adventure", 16: "Animation", 35:"Comedy", 80: "Crime", 99: "Documentary",
                 18: "Drama", 10751: "Family", 14: "Fantasy", 36: "History", 27: "Horror", 10402: "Music",
                 9648: "Mystery",10749: "Romance", 878: "Science Fiction", 10770: "TV Movie", 53: "Thriller",
                 10752: "War", 37: "Western"}

In [ ]:
tmdb["genre_names"] = tmdb["genre_ids"].map(lambda x: genre_id_dict[x])

In [ ]:
#Jeff's df
tn.head()